In [ ]:
import pandas as pd
import numpy as np
import gspread

# authentication might change
from google.auth import default
creds, _ = default()
gc = gspread.authorize(creds)

# A tool to get data from different worksheet in google sheets that goes numerically like xyz1(sh1,sh2,sh3...), 
# xyz2(sh1,sh2,sh3...) and so on
# run in google colab

df_concat = pd.DataFrame()
sheetname = 'sheetname'
worksheet = "sh"
L_number = 1
# a dict needed to map "xyz1" to int and +1 for loop
mapper = {'1':'1st', '2':'2nd', '3':'3rd', '4':'4th'}
spreadsheet_number = '1' 
list1 = []

# Looping
while L_number <=10 :
    # open tabs within a worksheet
    worksheet1 = gc.open(sheetname).worksheet('sh1') 
    worksheet2 = gc.open(sheetname).worksheet('sh2')
    # get_all_values gives a list of rows.
    rows1 = worksheet1.get_all_values()
    rows2 = worksheet2.get_all_values()
    #  Convert to a DataFrame
    df1 = pd.DataFrame.from_records(rows1)
    df2 = pd.DataFrame.from_records(rows2)

    # Fix the column and rows
    df1_header = df1.iloc[0]
    df1 = df1[1:]
    df1.columns = df1_header
    df1['ID'].replace('', np.nan, inplace=True) #replace empty space ' ' with np.nan then dropna
    df1['Name'].replace('', np.nan, inplace=True)
    
    df2_header = df2.iloc[0]
    df2 = df2[1:]
    df2.columns = df2_header
    df2['ID'].replace('', np.nan, inplace=True) 
    df2['Name'].replace('', np.nan, inplace=True)
    

    df_concat= pd.concat([df1, df2], ignore_index=True)
    df_concat.dropna(axis=0, how='all', inplace = True)

    # Identifier for different xyz(s), xyz1 should have a column to let user know it's 1
    # Column for identifier number 
    df_concat['identifier'] = 'identifier '+ spreadsheet_number
    # put df into list for some purposes not inlcuded in this code 
    list1.extend(df_concat.values.tolist())

    
    # Change str to int for increment
    spreadsheet_number_int = int(spreadsheet_number)    
    spreadsheet_number_int = spreadsheet_number_int+1
    L_number = L_number + 1
    if spreadsheet_number_int == 3:
        break
    spreadsheet_number_str= str(spreadsheet_number_int)  # change increment value into str for dict map
    # sheetname could be something like finance 1 and needs to be finance 2 for the loop
    # customizable here depending on sheetname
    split = sheetname.split()  # split input into parts and extract the last part
    split[1] = mapper[spreadsheet_number_str]  # assign new value using dict
    sheetname = " ".join(split) 
    spreadsheet_number = spreadsheet_number_str

In [ ]:
# a few issues was encountered when using this code and improvement in the future
# 1. in the data source, column name "ID"/"Name" was different a few times and has to be changed manually
    # implement something to check the datasource and change it to correct name if its wrong
# 2. use function/loop to handle the increasing xyz # and sh #
